## Connect to database

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"

conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")

cursor = conn.cursor()


## Code to Add All Subdirs of "Python_scripts" to sys.path

In [ ]:
import sys
from pathlib import Path

# Add Python_scripts to sys.path (parent of Extract_db_columns)
scripts_dir = Path().resolve().parents[0] / "Python_scripts"
sys.path.append(str(scripts_dir))

# Add the *project root* to sys.path (i.e., the parent of Python_scripts)
project_root = Path().resolve().parents[0]  # Adjust if notebook is nested deeper
sys.path.append(str(project_root))


In [ ]:
# # cursor.execute("DELETE FROM dlc_table WHERE id = %s;", (532,))

# # conn.commit()

# cursor.execute("""
#     UPDATE dlc_table
#     SET video_name = %s
#     WHERE id = %s;
# """, ('ToyLightInhibitory_1_22_25_S4P_P.mp4', 533))

# conn.commit()


## Insert video file names (Run this from the computer where the videos are located)

In [ ]:
from pathlib import Path

project_root = Path.home() / "Downloads" / "DeepLabCutProjects" / "data"
base_folder = project_root.joinpath("WhiteAnimals", "10X", "ToyLight", "SplitVideos")

print(f"Looking in: {base_folder}, Exists? {base_folder.exists()}")

# Collect video files
video_files = list(base_folder.rglob("*.mp4")) + list(base_folder.rglob("*.avi"))

# Create relative path or name
video_records = [(vf.name,) for vf in video_files]

# Get existing entries
cursor.execute("SELECT video_name FROM dlc_table;")
existing = set(row[0] for row in cursor.fetchall())

# Filter only new
new_records = [vr for vr in video_records if vr[0] not in existing]

# Insert
if new_records:
    insert_query = "INSERT INTO dlc_table (video_name) VALUES (%s);"
    cursor.executemany(insert_query, new_records)
    conn.commit()
    print(f"Inserted {len(new_records)} new videos.")
else:
    print("No new videos to insert.")


## Insert task, date_str, name, health, id_ in dlc_table

In [ ]:
import importlib
import Extract_db_columns.parse_video_name
importlib.reload(Extract_db_columns.parse_video_name)
from Extract_db_columns.parse_video_name import parse_video_name

cursor.execute("SELECT id, video_name FROM dlc_table WHERE video_name IS NOT NULL AND task IS NULL;")
rows = cursor.fetchall()

updates = []

for row in rows:
    id_, video_name = row
    task, date_str, name, health = parse_video_name(video_name)
    updates.append((task, date_str, name, health, id_))

# Update in batch
cursor.executemany("""
UPDATE dlc_table
SET task = %s, date = %s, name = %s, health = %s
WHERE id = %s;
""", updates)

conn.commit()


## Insert num_frames, frame_rate, video_width, video_height

In [ ]:
import importlib
import Extract_db_columns.update_video_info

importlib.reload(Extract_db_columns.update_video_info)
from Extract_db_columns.update_video_info import update_video_info_in_db

# Call the function with subdirectory
base_video_dir = Path.home() / "Downloads" / "DeepLabCutProjects" / "data"
video_subdir = "WhiteAnimals/10X/ToyLight"

update_video_info_in_db(conn, base_video_dir, video_subdir)


## Insert trial_length

In [ ]:
import importlib
import Extract_db_columns.insert_trial_length

importlib.reload(Extract_db_columns.insert_trial_length)
from Extract_db_columns.insert_trial_length import insert_trial_length

query = "SELECT id FROM dlc_table WHERE trial_length IS NULL ORDER BY id"

df = pd.read_sql_query(query, conn)
id_list = df['id'].to_list()

insert_trial_length(id_list, conn)


## Insert genotype

In [ ]:
cursor = conn.cursor()
cursor.execute("UPDATE dlc_table SET genotype = 'white' WHERE genotype is NULL")
conn.commit()


## Insert maze_number

In [ ]:
from Extract_db_columns.extract_maze_number import (
    load_mother_videos, build_prefix_to_animal_map, get_maze_number, update_maze_numbers_in_db
)

raw_video_dirs = [
    "/Users/atanugiri/Downloads/DeepLabCutProjects/data/WhiteAnimals/10X/LightOnly/RawVideos"
]

mother_videos = load_mother_videos(raw_video_dirs)
prefix_to_animals = build_prefix_to_animal_map(mother_videos)

update_maze_numbers_in_db(conn, "dlc_table", prefix_to_animals)


## Insert csv_file_path

In [ ]:
import importlib
from tqdm import tqdm

import Python_scripts.Extract_db_columns.find_csv_for_video as find_mod
importlib.reload(find_mod)
from Python_scripts.Extract_db_columns.find_csv_for_video import find_csv_for_video

# Query for 'FoodOnly%' tasks
query = """
SELECT id, video_name
FROM dlc_table
WHERE task='LightOnly' AND genotype='white' AND frame_rate IS NOT NULL AND dose_mult=10
ORDER BY id;
"""
df = pd.read_sql_query(query, conn)
# df.to_csv("dlc_table_FoodOnly.csv", index=False)

# Directory to search
dir_to_search = (Path.home() / "Downloads" / "DeepLabCutProjects" / "data" / "WhiteAnimals" / 
                  "10X" / "LightOnly" / "DlcDataPytorchFiltered")

# Loop and update
for _, row in tqdm(df.iterrows(), total=len(df), desc="Inserting CSV path for LightOnly"):
    video_id = row["id"]
    video_name = row["video_name"]
    csv_path = find_csv_for_video(video_name, dir_to_search)
    print(csv_path)

    if csv_path:
        cursor.execute(
            "UPDATE dlc_table SET csv_file_path = %s WHERE id = %s;",
            (csv_path, video_id)
        )
    else:
        print(f"❌ No CSV found for {video_name}")

conn.commit()


## Add modulation column

In [ ]:
SQLS = [
    "ALTER TABLE dlc_table ADD COLUMN IF NOT EXISTS modulation TEXT;",
    """
    UPDATE dlc_table
    SET modulation = 'Inhibitory',
        task = TRIM(REPLACE(task, 'Inhibitory', ''))
    WHERE task ILIKE '%Inhibitory';
    """,
    """
    UPDATE dlc_table
    SET modulation = 'Excitatory',
        task = TRIM(REPLACE(task, 'Excitatory', ''))
    WHERE task ILIKE '%Excitatory';
    """,
    "UPDATE dlc_table SET modulation = 'NA' WHERE modulation IS NULL;",
]

try:
    for stmt in SQLS:
        cursor.execute(stmt)
    conn.commit()
    print("Done.")


## Add dose_mult

In [ ]:
SQL = "UPDATE dlc_table SET dose_mult=10 WHERE dose_mult IS NULL"
cursor.execute(SQL)
conn.commit()


### Completely Clear Table First (CAREFUL)

In [ ]:
cursor.execute("DELETE FROM dlc_files")
conn.commit()
print("All entries deleted from dlc_files.")
# conn.rollback()